## ワークスペース設定

In [22]:
import azure.ai.ml 
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azureml.core import Workspace

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')


try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    # This will open a browser page for
    credential = InteractiveBrowserCredential()

try:
    ml_client = MLClient.from_config(credential=credential)
except Exception as ex:
    # NOTE: Update following workspace information if not correctly configure before
    client_config = {
        "subscription_id": ws.subscription_id,
        "resource_group": ws.resource_group,
        "workspace_name": ws.name,
    }

    if client_config["subscription_id"].startswith("<"):
        print(
            "please update your <SUBSCRIPTION_ID> <RESOURCE_GROUP> <AML_WORKSPACE_NAME> in notebook cell"
        )
        raise ex
    else:  # write and reload from config file
        import json, os

        config_path = "../.azureml/config.json"
        os.makedirs(os.path.dirname(config_path), exist_ok=True)
        with open(config_path, "w") as fo:
            fo.write(json.dumps(client_config))
        ml_client = MLClient.from_config(credential=credential, path=config_path)
print(ml_client)

Found the config file in: /mnt/batch/tasks/shared/LS_root/mounts/clusters/skilling-ml2/code/Users/localdev/mslearn/train-evaluate-regression-models/.azureml/config.json


## 実験ジョブの実行

### 入力データの呼び出し

In [23]:

registered_data_asset = ml_client.data._get_latest_version(name="daily-bike-share")


### 実行

In [25]:
import time 

from azure.ai.ml import command, Input, Output
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes


experiment_name = "mslean-train-evaluate-regression-models" 
run_name = f'job_sklearn_run_{int(time.time())}' # 実行ごとにかぶらないようにしつつ好きな名前を指定

my_job_inputs = {
    "input_data": Input(type=AssetTypes.URI_FILE, path=registered_data_asset.id)
}

job = command(
    code="./src",
    command="python main.py --input_data ${{inputs.input_data}}",
    inputs=my_job_inputs,
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu:9",
    compute="cpu-cluster",
    experiment_name=experiment_name,
    display_name=run_name,
)

# submit the command
returned_job = ml_client.create_or_update(job)
# get a URL for the status of the job
returned_job.services["Studio"].endpoint

Uploading src (0.01 MBs): 100%|██████████| 5610/5610 [00:00<00:00, 47124.46it/s]




'https://ml.azure.com/runs/joyful_longan_flswj1x1sp?wsid=/subscriptions/aa7533c4-42d3-4f74-9b7e-9bec680a7c22/resourcegroups/vse-rg-sand/workspaces/skilling-ml-sand&tid=8942ec59-80b8-4a4c-b002-32b6193ca367'

In [28]:

registered_data_asset_v1 = ml_client.data._get_latest_version(name="daily-bike-share-sdkv1_1")


In [29]:
registered_data_asset_v1

Data({'type': 'mltable', 'is_anonymous': False, 'auto_increment_version': False, 'name': 'daily-bike-share-sdkv1_1', 'description': None, 'tags': {}, 'properties': {}, 'id': '/subscriptions/aa7533c4-42d3-4f74-9b7e-9bec680a7c22/resourceGroups/vse-rg-sand/providers/Microsoft.MachineLearningServices/workspaces/skilling-ml-sand/data/daily-bike-share-sdkv1_1/versions/1', 'base_path': './', 'creation_context': <azure.ai.ml._restclient.v2022_05_01.models._models_py3.SystemData object at 0x7f530558d280>, 'serialize': <msrest.serialization.Serializer object at 0x7f53057ebfd0>, 'version': '1', 'latest_version': None, 'path': 'azureml://subscriptions/aa7533c4-42d3-4f74-9b7e-9bec680a7c22/resourcegroups/vse-rg-sand/workspaces/skilling-ml-sand/datastores/workspaceblobstore/paths/datasets/daily-bike-share-sdkv2/03d14e3c-c254-4a0a-a01a-98c2db3223bd/', 'referenced_uris': None})

In [31]:
import time 

from azure.ai.ml import command, Input, Output
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes


experiment_name = "mslean-train-evaluate-regression-models" 
run_name = f'assetv1_job_run_{int(time.time())}' # 実行ごとにかぶらないようにしつつ好きな名前を指定

my_job_inputs = {
    "input_data": Input(
        type=AssetTypes.MLTABLE,  
        path=registered_data_asset_v1.id,
        mode="eval_mount")
    
}

job = command(
    code="./src",
    command="python main.py --input_data ${{inputs.input_data}}",
    inputs=my_job_inputs,
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu:9",
    compute="cpu-cluster",
    experiment_name=experiment_name,
    display_name=run_name,
)

# submit the command
returned_job = ml_client.create_or_update(job)
# get a URL for the status of the job
returned_job.services["Studio"].endpoint

Uploading src (0.01 MBs): 100%|██████████| 5638/5638 [00:00<00:00, 44336.65it/s]




'https://ml.azure.com/runs/serene_wing_1ft96cylq0?wsid=/subscriptions/aa7533c4-42d3-4f74-9b7e-9bec680a7c22/resourcegroups/vse-rg-sand/workspaces/skilling-ml-sand&tid=8942ec59-80b8-4a4c-b002-32b6193ca367'